In [7]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D,Flatten,Dense,Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np

(x_train,y_train),(x_test,y_test)=fashion_mnist.load_data()
x_train=x_train.reshape((-1,28,28,1)).astype('float32')/255.0
x_test=x_test.reshape((-1,28,28,1)).astype('float32')/255.0
y_train=to_categorical(y_train,10)
y_test=to_categorical(y_test,10)

#Split the data into training and validation sets
x_train,x_val,y_train,y_val=train_test_split(x_train,y_train,test_size=0.2,random_state=42)

model=Sequential([
    Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(28,28,1)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32,kernel_size=(3,3),activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dropout(0.3),
    Dense(128,activation='relu'),
    Dense(10,activation='softmax')
])

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(x_train,y_train,epochs=10,batch_size=32,validation_data=(x_val,y_val))



# ✅ Export for TensorFlow Serving
export_path = "C:/Users/ADARSH/Downloads/saved_model/1"
model.export(export_path)
print(f"Model exported to {export_path}")


c:\Users\ADARSH\.conda\envs\tf_env\lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.7325 - loss: 0.7302 - val_accuracy: 0.8638 - val_loss: 0.3773
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.8601 - loss: 0.3866 - val_accuracy: 0.8820 - val_loss: 0.3175
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.8775 - loss: 0.3347 - val_accuracy: 0.8881 - val_loss: 0.3035
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 14s 9ms/step - accuracy: 0.8882 - loss: 0.2977 - val_accuracy: 0.8938 - val_loss: 0.2837
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.8956 - loss: 0.2761 - val_accuracy: 0.9051 - val_loss: 0.2676
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.9041 - loss: 0.2545 - val_accuracy: 0.9017 - val_loss: 0.2616
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 9ms/step - accuracy: 0.9086 - loss: 0.2423 - val_accuracy: 0.9047 - val_loss: 0.2564
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 13s 8ms/step - accuracy: 0.9109 - loss: 

INFO:tensorflow:Assets written to: C:/Users/ADARSH/Downloads/saved_model/1\assets


Saved artifact at 'C:/Users/ADARSH/Downloads/saved_model/1'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='keras_tensor_8')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  2404190283808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2404190382992: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2404190386160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2404190387744: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2404190387920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2404190474432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2404190470560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  2404190470384: TensorSpec(shape=(), dtype=tf.resource, name=None)
Model exported to C:/Users/ADARSH/Downloads/saved_model/1


In [17]:
import requests, json
import numpy as np

data = {
    "signature_name": "serving_default",
    "instances": np.random.rand(1, 28, 28).tolist()
}

response = requests.post("http://localhost:8501/v1/models/saved_model:predict", json=data)
print(response.json())


{'predictions': [[0.0901232213, 0.0798598751, 0.120819032, 0.0951336548, 0.094939515, 0.102226026, 0.0980775654, 0.0753462389, 0.150925145, 0.092549704]]}


In [18]:
import numpy as np

pred = [0.0860122517, 0.0798024386, 0.116337605, 0.0987222791, 0.0990732089,
        0.0904200524, 0.0970608667, 0.0825301707, 0.152064666, 0.097976476]
predicted_class = np.argmax(pred)
print(f"Predicted digit: {predicted_class}")


Predicted digit: 8


docker rm tf_model_serving_v2


docker run -p 8501:8501 ^

  --name=tf_model_serving_v2 ^

  --mount type=bind,source="C:/Users/ADARSH/Downloads/saved_model",target=/models/saved_model ^

  -e MODEL_NAME=saved_model ^

  -t tensorflow/serving
